# PREDICTING LOAN REPAYMENT

In [27]:
loans = read.csv("loans.csv")

In [28]:
str(loans)

'data.frame':	9578 obs. of  14 variables:
 $ credit.policy    : int  1 1 1 1 1 1 1 1 1 1 ...
 $ purpose          : Factor w/ 7 levels "all_other","credit_card",..: 3 2 3 3 2 2 3 1 5 3 ...
 $ int.rate         : num  0.119 0.107 0.136 0.101 0.143 ...
 $ installment      : num  829 228 367 162 103 ...
 $ log.annual.inc   : num  11.4 11.1 10.4 11.4 11.3 ...
 $ dti              : num  19.5 14.3 11.6 8.1 15 ...
 $ fico             : int  737 707 682 712 667 727 667 722 682 707 ...
 $ days.with.cr.line: num  5640 2760 4710 2700 4066 ...
 $ revol.bal        : int  28854 33623 3511 33667 4740 50807 3839 24220 69909 5630 ...
 $ revol.util       : num  52.1 76.7 25.6 73.2 39.5 51 76.8 68.6 51.1 23 ...
 $ inq.last.6mths   : int  0 0 1 1 0 0 0 0 1 1 ...
 $ delinq.2yrs      : int  0 0 0 0 1 0 0 0 0 0 ...
 $ pub.rec          : int  0 0 0 0 0 0 1 0 0 0 ...
 $ not.fully.paid   : int  0 0 0 0 0 0 1 1 0 0 ...


In [29]:
table(loans$not.fully.paid)


   0    1 
8045 1533 

In [30]:
1533/(1533+8045)

[1] 0.1600543

In [31]:
summary(loans)

 credit.policy                 purpose        int.rate       installment    
 Min.   :0.000   all_other         :2331   Min.   :0.0600   Min.   : 15.67  
 1st Qu.:1.000   credit_card       :1262   1st Qu.:0.1039   1st Qu.:163.77  
 Median :1.000   debt_consolidation:3957   Median :0.1221   Median :268.95  
 Mean   :0.805   educational       : 343   Mean   :0.1226   Mean   :319.09  
 3rd Qu.:1.000   home_improvement  : 629   3rd Qu.:0.1407   3rd Qu.:432.76  
 Max.   :1.000   major_purchase    : 437   Max.   :0.2164   Max.   :940.14  
                 small_business    : 619                                    
 log.annual.inc        dti              fico       days.with.cr.line
 Min.   : 7.548   Min.   : 0.000   Min.   :612.0   Min.   :  179    
 1st Qu.:10.558   1st Qu.: 7.213   1st Qu.:682.0   1st Qu.: 2820    
 Median :10.928   Median :12.665   Median :707.0   Median : 4140    
 Mean   :10.932   Mean   :12.607   Mean   :710.8   Mean   : 4562    
 3rd Qu.:11.290   3rd Qu.:17.950   3rd 

In [32]:
library("mice")

In [33]:
set.seed(144)

In [34]:
vars.for.imputation = setdiff(names(loans), "not.fully.paid")

In [35]:
imputed = complete(mice(loans[vars.for.imputation]))


 iter imp variable
  1   1  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  1   2  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  1   3  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  1   4  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  1   5  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  2   1  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  2   2  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  2   3  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  2   4  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  2   5  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  3   1  log.annual.inc  days.with.cr.line  revol.

In [36]:
loans[vars.for.imputation] = imputed

In [37]:
loans_imputed = read.csv("loans_imputed.csv")

In [38]:
# #Splitting the dataset into Training and Testing data
library(caTools)

In [39]:
split = sample.split(loans_imputed$not.fully.paid, SplitRatio = 0.7)

In [40]:
train = subset(loans_imputed, split == TRUE)
test = subset(loans_imputed, split == FALSE)

In [41]:
# #Logistic Regression
model_1 = glm(not.fully.paid ~ ., data = train, family = "binomial")

In [42]:
summary(model_1)


Call:
glm(formula = not.fully.paid ~ ., family = "binomial", data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.9757  -0.6171  -0.4953  -0.3608   2.4748  

Coefficients:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)                7.438e+00  1.541e+00   4.826 1.39e-06 ***
credit.policy             -2.940e-01  1.011e-01  -2.909 0.003631 ** 
purposecredit_card        -5.906e-01  1.323e-01  -4.463 8.08e-06 ***
purposedebt_consolidation -4.037e-01  9.225e-02  -4.376 1.21e-05 ***
purposeeducational         1.759e-01  1.758e-01   1.000 0.317104    
purposehome_improvement   -5.255e-02  1.575e-01  -0.334 0.738590    
purposemajor_purchase     -5.193e-01  2.064e-01  -2.516 0.011862 *  
purposesmall_business      4.832e-01  1.392e-01   3.472 0.000516 ***
int.rate                   3.074e+00  2.029e+00   1.515 0.129755    
installment                1.198e-03  2.090e-04   5.734 9.83e-09 ***
log.annual.inc            -3.854e-01

In [44]:
-10*-7.953e-03

[1] 0.07953

In [48]:
exp(0.07953)

[1] 1.082778

In [45]:
# #PREDICTION

In [46]:
test$predicted.risk = predict(model_1, newdata = test, type = "response")

In [47]:
table(test$not.fully.paid, test$predicted.risk > 0.5)

   
    FALSE TRUE
  0  2406    7
  1   438   22

In [51]:
library(ROCR)

In [52]:
pred = prediction(test$predicted.risk, test$not.fully.paid)

In [53]:
as.numeric(performance(pred, "auc")@y.values)

[1] 0.6745716